# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [ ]:
# Downloading the Digital Rock Physics Benchmarks dataset.
!git clone https://github.com/fkrzikalla/drp-benchmarks

# Removing unnecessary files.
!rm -r drp-benchmarks/images/fontainebleau
!rm -r drp-benchmarks/images/spherepack

!rm drp-benchmarks/images/*/segmented-kongju.mat
!rm drp-benchmarks/images/*/segmented-stanford.raw.gz
!rm drp-benchmarks/images/*/segmented-vsg.raw.gz

# Uncompressing raw segmentations.
!gunzip drp-benchmarks/images/berea/segmented-kongju.raw.gz
!gunzip drp-benchmarks/images/grosmont/segmented-kongju.raw.gz

In [ ]:
!pip install torchinfo

In [3]:
# Basic imports.
import gc
import os
import time
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models

from skimage import io
from skimage import transform

from sklearn import metrics

from matplotlib import pyplot as plt
from torchinfo import summary

from tqdm.notebook import tqdm

%matplotlib inline

cudnn.benchmark = True

In [ ]:
# Setting predefined arguments.
args = {
    'epoch_num': 10,                    # Number of epochs.
    'dataset': 'grosmont',              # Dataset. (bereau or grosmont)
    'n_classes': 2,                     # Number of classes in segmentation task.
    'lr': 1e-4,                         # Learning rate.
    'weight_decay': 5e-4,               # L2 penalty.
    'momentum': 0.9,                    # Momentum.
    'num_workers': 2,                   # Number of workers on data loader.
    'batch_size': 8,                    # Mini-batch size.
    'crop_size': 256,                   # Random crop size.
    'show_freq': 1,                     # Frequency for showing predictions.
    'root': 'drp-benchmarks/images'     # root directory for the images
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

In [5]:
# Random initialization for weights and biases.
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()

In [6]:
def evaluate(prds, labs, num_classes):

    iou_list = []
    f1_list = []

    for prd, lab in zip(prds, labs):

        iou = metrics.jaccard_score(lab.ravel(), prd.ravel())
        f1 = metrics.f1_score(lab.ravel(), prd.ravel())

        iou_list.append(iou)
        f1_list.append(f1)

    iou_list = np.asarray(iou_list)
    f1_list = np.asarray(f1_list)

    return iou_list, f1_list

# Introdução

Neste notebook faremos alguns experimentos com o Dataset [DRP-Benchmarks](https://www.sciencedirect.com/science/article/pii/S0098300412003147) sendo utilizados as bases de Bereau e Grosmont. Em seguida treinaremos uma UNet para realizar a segmentação semântica das rochas nas imagens extraidas.

# Dataloaders Customizados

Os frameworks de Deep Learning modernos (i.e. MXNet e Pytorch) permitem a criação de dataloaders customizados ao se sobrescreverem classes desses frameworks. Esse tipo de dataloader é especialmente útil no caso de tarefas diferentes das de classificação que temos visto até agora (i.e. segmentação e detecção de imagens, processamento de áudio, processamento de linguagem, natural, etc), nas quais os labels podem ser mais esparsos ou densos que rótulos de classificação.

Usando como base as classes [*Dataloader*](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) e [*Dataset*](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) do subpacote [*data*](https://pytorch.org/docs/stable/data.html) do Pytorch, podemos customizar a leitura dos dados ao mesmo tempo em que paralelizamos a leitura das amostras dos nossos batches. A paralelização da leitura de amostras em várias [threads](https://www.tutorialspoint.com/python/python_multithreading) torna o uso da GPU o mais eficiente possível, já que não é necessário deixar a GPU esperando pelo carregamento de novas amostras para compor um batch.

In [7]:
# Class that reads the DRP Benchmarks dataset.
class RockDataset(data.Dataset):

    def __init__(self, root, mode, dataset, crop_size=256, num_classes=2):

        # Initializing variables.
        self.root = root
        self.mode = mode
        self.dataset = dataset
        self.crop_size = crop_size
        self.num_classes = num_classes

        # Creating list of image files.
        self.imgs = self.make_dataset()

        # Check for consistency in list.
        if len(self.imgs) == 0:

            raise (RuntimeError('Found 0 images, please check the data set.'))

    def make_dataset(self):

        # Joining input paths.
        self.img_path = os.path.join(self.root,
                                     self.dataset,
                                     'tif')
        self.msk_path = os.path.join(self.root,
                                     self.dataset,
                                     'segmented-kongju.raw')

        # Preload mask volume.
        transpose_sequence = [0, 2, 1]

        if self.mode == 'train':
            self.msk_vol = np.fromfile(open(self.msk_path, 'rb'),
                                       dtype=np.int8).reshape(1024, 1024, 1024)
            self.msk_vol = self.msk_vol.transpose(*transpose_sequence)[:-64]

        elif self.mode == 'test':
            self.msk_vol = np.fromfile(open(self.msk_path, 'rb'),
                                       dtype=np.int8).reshape(1024, 1024, 1024)
            self.msk_vol = self.msk_vol.transpose(*transpose_sequence)[-64:]

        # Reading paths from file.
        if self.mode == 'train':
            items = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[:-64]
        elif self.mode == 'test':
            items = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[-64:]

        # Returning list.
        return items

    def __getitem__(self, index):

        # Reading items from list.
        f = self.imgs[index]

        img_path = os.path.join(self.img_path, f)

        # Reading images.
        img = io.imread(img_path)
        msk = self.msk_vol[index]

        # Casting images to the appropriate dtypes.
        img = img.astype(np.float32)
        msk = msk.astype(np.int64)

        # Normalization (z-score).
        img = (img - img.mean()) / img.std()

        # Random cropping images.
        if self.mode == 'train':

            rand_hw = np.random.randint(0, 1024 - self.crop_size, size=2)

            img = img[rand_hw[0]:(rand_hw[0] + self.crop_size),
                      rand_hw[1]:(rand_hw[1] + self.crop_size)]
            msk = msk[rand_hw[0]:(rand_hw[0] + self.crop_size),
                      rand_hw[1]:(rand_hw[1] + self.crop_size)]

            # Fixing channel dimension.
            img = np.expand_dims(img, axis=0) # From (H, W) to (C=1, H, W).

        if self.mode == 'test':

            img = [img[0:256, 0:256],    img[256:512, 0:256],    img[512:768, 0:256],    img[768:1024, 0:256],
                   img[0:256, 256:512],  img[256:512, 256:512],  img[512:768, 256:512],  img[768:1024, 256:512],
                   img[0:256, 512:768],  img[256:512, 512:768],  img[512:768, 512:768],  img[768:1024, 512:768],
                   img[0:256, 768:1024], img[256:512, 768:1024], img[512:768, 768:1024], img[768:1024, 768:1024],
                  ]
            msk = [msk[0:256, 0:256],    msk[256:512, 0:256],    msk[512:768, 0:256],    msk[768:1024, 0:256],
                   msk[0:256, 256:512],  msk[256:512, 256:512],  msk[512:768, 256:512],  msk[768:1024, 256:512],
                   msk[0:256, 512:768],  msk[256:512, 512:768],  msk[512:768, 512:768],  msk[768:1024, 512:768],
                   msk[0:256, 768:1024], msk[256:512, 768:1024], msk[512:768, 768:1024], msk[768:1024, 768:1024],
                  ]

            img = np.asarray(img, dtype=np.float32)
            msk = np.asarray(msk, dtype=np.int64)

        # Turning numpy arrays to tensors.
        img = torch.from_numpy(img)
        msk = torch.from_numpy(msk)

        # Returning to iterator.
        return img, msk

    def __len__(self):

        return len(self.imgs)

gc.collect()

# Instantiating dataloaders.
root = args['root']

# Setting datasets.
train_set = RockDataset(root,
                        'train',
                        args['dataset'],
                        args['crop_size'])
test_set = RockDataset(root,
                       'test',
                       args['dataset'],
                       args['crop_size'])

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          batch_size=args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         batch_size=1,
                         num_workers=args['num_workers'],
                         shuffle=False)

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(16, 8))
for i, batch_data in enumerate(train_set):

    if i >= 4:
        break

    img, lab = batch_data


    ax[0][i].imshow(img[0].numpy(), cmap=plt.get_cmap('gray'))
    # ax[0][i].imshow(lab.numpy(), 'Greens_r', interpolation='nearest', alpha=0.3) # Uncomment to see overlay of labels.
    ax[0][i].set_yticks([])
    ax[0][i].set_xticks([])

    ax[1][i].imshow(lab.numpy(), cmap=plt.get_cmap('gray'))
    ax[1][i].set_yticks([])
    ax[1][i].set_xticks([])

plt.show()

# Atividade Prática: Implementando pipeline de segmentação

Usar as arquiteturas de segmentação estudadas durante essa semana e adaptá-la para o dataset de rochas. O dataset contém apenas duas classes e um canal de input, tendo o dataloader já pré-definido a cima. Para essa tarefa vamos usar uma arquitetura UNet.

Os passos a serem seguidos são os seguintes:

1.   Definir arquitetura de uma rede de segmentação.
2.   Definir uma loss.
3.   Instanciar um otimizador.
4.   Implementar funções de *train()* e *test()*.
5.   Implementar for que itera sobre as epochs.

Primeiramente vamos implementar a arquitetura da UNet. Para isso, vamos dividir a arquitetura em blocos "encoder" e blocos "decoder". Os encoders são usados na parte inicial da rede, e os decoders na parte final.

Cada bloco encoder possui duas camadas de convolução com `kernel_size=3` e `padding=1`, de forma com que essas convoluções não alterem as dimensões espaciais (altura $H$ e largura $W$) da entrada. Apenas no final de cada bloco encoder temos um `max_pooling` para reduzir as dimensões espaciais da entrada (ela terá metade da altura e largura).

Cada bloco decoder possui duas camadas de convolução, da mesma forma dos encoders (sem alterar dimensões espaciais). No final do bloco, para aumentar a dimensão espacial da entrada, ele possui uma convolução transposta com `kernel_size=2` e `stride=2` para dobrar a altura e largura da entrada.

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self, in_channels, out_channels):

        super(EncoderBlock, self).__init__()

        # TODO: each encoder block be composed of:
        #        1) 2d conv inputting in_channels and outputting out_channels;
        #        2) batch norm;
        #        3) ReLU;
        #        4) 2d conv outputting out_channels;
        #        5) batch norm;
        #        6) ReLU;
        #        7) max pooling 2d with kernel 2x2 and stride of 2.
        self.encode = nn.Sequential(
            ...
        )

    def forward(self, x):

        # TODO: implement forward on self.encode.
        return ...


class DecoderBlock(nn.Module):

    def __init__(self, in_channels, middle_channels, out_channels):

        super(DecoderBlock, self).__init__()

        # TODO: each decoder block be composed of:
        #        1) channel dropout;
        #        2) 2d conv inputting in_channels and outputting
        #           middle_channels;
        #        3) batch norm;
        #        4) 2d conv outputting middle_channels;
        #        5) batch norm;
        #        6) ReLU;
        #        7) transposed 2d conv outputting out_channels with kernel 2x2,
        #           stride of 2, padding and output_padding of 0.
        self.decode = nn.Sequential(
            ...
        )

    def forward(self, x):

        # TODO: implement forward on self.decode.
        return ...

class UNet(nn.Module):

    def __init__(self, input_channels, num_classes):

        super(UNet, self).__init__()

        # TODO: this U-Net will be composed of 3 Encoder Blocks and 3
        #        Decoder Blocks:
        #        1) Encoder Block inputting input_channels and outputting 32
        #           channels;
        #        2) Encoder Block outputting 64 channels;
        #        3) Encoder Block 128 channels;
        #        4) Decoder Block inputting 128 channels, 256 middle channels
        #           and outputting 128 channels;
        #        5) Decoder Block inputting 256 channels, 128 middle channels
        #           and outputting 64 channels;
        #        6) Decoder Block inputting 128 channels, 64 middle channels
        #           and outputting 32 channels;
        #        7) channel dropout;
        #        8) 2d conv block (conv + batch norm + ReLU) inputting 64 channels
        #           and outputting 32 channels;
        #        9) 2d conv block (conv + batch norm + ReLU) inputting 32 channels
        #           and outputting 32 channels;
        #        10) 2d conv block with kernel 1x1 outputting num_classes
        #            channels. This is the pixel classification layer.
        pass

        # Random initialization for weights.
        initialize_weights(self)

    def forward(self, x):

        # TODO: implement forward on Encoder Blocks.
        ...

        # TODO: implement forward on Decoder Blocks. Each Decoder receives the
        #        output of the last layer concatenated with the output of an
        #        Encoder Block with compatible size. See U-Net architectural
        #        scheme above.
        ...

        # TODO: implement forward on the final 2 conv blocks and final 1x1
        #        conv to predict classes.
        ...

        # TODO: return output.
        return ...

net = UNet(1, num_classes=args['n_classes']).to(args['device'])

summary(net, (1, 1, 256, 256))

In [11]:
# Setting optimizer.
optimizer = optim.Adam([p for p in list(net.parameters()) if p.requires_grad],
                       lr=args['lr'],
                       weight_decay=args['weight_decay'],
                       betas=(args['momentum'], 0.999))

In [12]:
# TODO: Setting loss.
criterion = ...

In [13]:
# Training function.
def train(train_loader, net, criterion, optimizer, epoch):

    tic = time.time()

    # Setting network for training mode.
    net.train()

    # Average Meter for batch loss.
    train_loss = []

    # Lists for whole epoch loss.
    labs_all, prds_all = [], []

    # Iterating over batches.
    for i, batch_data in (pbar := tqdm(enumerate(train_loader), total=len(train_loader), unit='batch')):
        # Obtaining images and labels for batch.
        inps, labs = batch_data

        # TODO: Casting to cuda variables (inps and labs).
        inps = ...
        labs = ...

        # TODO: Clearing the gradients of optimizer.
        ...

        # TODO: Forwarding.
        outs = ...

        # TODO: Computing loss.
        loss = ...

        # TODO: Computing backpropagation and taking optimizer step.
        ...

        # Obtaining predictions.
        prds = outs.data.max(1)[1].squeeze_(1).squeeze(0).cpu().numpy()

        # Appending images for epoch loss calculation.
        labs_all.append(labs.detach().data.squeeze(0).cpu().numpy())
        prds_all.append(prds)

        # Updating loss meter.
        train_loss = np.append(train_loss, loss.data.item())

        pbar.set_description(f"Loss: {train_loss.mean():.4f}")

    # Computing error metrics for whole epoch.
    iou, f1 = evaluate(prds_all, labs_all, args['n_classes'])
    toc = time.time()

    # Printing training epoch loss and metrics.
    # print('-------------------------------------------------------------------')
    print('[train], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        train_loss.mean(), train_loss.std(), iou.mean(), iou.std(), f1.mean(), f1.std(), (toc - tic)))
    # print('-------------------------------------------------------------------')

def test(test_loader, net, criterion, epoch):

    tic = time.time()

    # Setting network for evaluation mode.
    net.eval()

    # Average Meter for batch loss.
    test_loss = []

    # Lists for whole epoch loss.
    labs_all, prds_all = [], []

    with torch.no_grad():

        # Iterating over batches.
        for i, batch_data in enumerate(test_loader):

            # Obtaining images and labels for batch.
            inps, labs = batch_data

            inps = inps.view(inps.size(1), 1, inps.size(2), inps.size(3))
            labs = labs.view(labs.size(1), labs.size(2), labs.size(3))

            # TODO: Casting to cuda variables.
            inps = ...
            labs = ...

            # TODO: Forwarding.
            outs = ...

            # TODO: Computing loss.
            loss = ...

            # Obtaining predictions.
            prds = outs.data.max(1)[1].squeeze_(1).squeeze(0).cpu().numpy()

            # Appending images for epoch loss calculation.
            labs_all.append(labs.detach().data.squeeze(0).cpu().numpy())
            prds_all.append(prds)

            # Updating loss meter.
            test_loss = np.append(test_loss, loss.data.item())

            # Showing some example results
            if i == 0 and epoch % args['show_freq'] == 0:

                fig, ax = plt.subplots(2, 3, figsize=(10, 7))

                perm = np.random.permutation(inps.size(0))

                for p in range(2):

                    ax[p, 0].imshow(inps[perm[p], 0].detach().cpu().numpy().squeeze())
                    ax[p, 0].set_yticks([])
                    ax[p, 0].set_xticks([])
                    ax[p, 0].set_title('Image')

                    ax[p, 1].imshow(labs[perm[p]].detach().data.squeeze(0).cpu().numpy().squeeze())
                    ax[p, 1].set_yticks([])
                    ax[p, 1].set_xticks([])
                    ax[p, 1].set_title('Label')

                    ax[p, 2].imshow(prds[perm[p]].squeeze())
                    ax[p, 2].set_yticks([])
                    ax[p, 2].set_xticks([])
                    ax[p, 2].set_title('Prediction')


    # Computing error metrics for whole epoch.
    iou, f1 = evaluate(prds_all, labs_all, args['n_classes'])
    toc = time.time()

    # Printing test epoch loss and metrics.
    # print('-------------------------------------------------------------------')
    print('[test], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        test_loss.mean(), test_loss.std(), iou.mean(), iou.std(), f1.mean(), f1.std(), (toc - tic)))

    plt.show()
    # print('-------------------------------------------------------------------')

In [ ]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):
    print(f' ========== Epoch {epoch} ========== ')

    # Training function.
    train(train_loader, net, criterion, optimizer, epoch)

    # Computing test loss and metrics.
    test(test_loader, net, criterion, epoch)